<h1> Document Vector Embeddings </h1>

Initial experiment will be perfomed based on the experiment by Sugathadasa et al. [https://arxiv.org/pdf/1805.10685.pdf]

<h2> Text Preprocessing </h2>

First step is to create a <i> document corpus </i> which is a subset of most important sentences in each document. We can do that by implementing the <i> PageRank </i> algorithm. Before we do that, we need to preprocess the document by cleaning the text of unwanted charachters and common words. We used lemmatization and case-folding to lowercase as first steps in cleaning the documents. 

<h5> Required libraries </h5>


In [ ]:
# !pip install requests

In [13]:
import requests
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import spacy
import nltk.data
import pandas as pd
from text_rank import analyze 
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import en_core_web_sm
import time
import os
import json
import ast
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [2]:
NUM_OF_DOCUMENTS = 5
NUM_OF_SENTENCES = 50
NUM_OF_CHARACTERS = 10

URL = "https://www.courtlistener.com/api/rest/v3/opinions/"

In [3]:
def get_document(file_name):
    data = ""
    with open(file_name) as json_file:
        data = json.load(json_file)
    return data["plain_text"].replace("\n", " ")


<h2> <i> TextRank </i> algorithm </h2>

<i> TextRank </i> algorithm will be implemented based on the work of Mihalcea et al [https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf]. <br>
We use this algorithm for extracting "most valuable" sentences in a document.  <br> <br>
<i> TextRank </i> algorithm is implemented in a python script named <i> text_rank.py </i>

In [4]:
def apply_textrank(text):
    sorted_sentences = analyze(text, NUM_OF_SENTENCES)
    return sorted_sentences

Sentences that are shorter than N characters should be removed.

<h2> Text processing after <i> TextRank </i> algorithm </h2>

After the <i> TextRank </i> algorithm we apply lemmatization to each word in the document

In [5]:
# Lemma Tokenizer called by TfIdfVectorizer

class LemmaTokenizer():
    def __init__(self):
        self.spacynlp = spacy.load('en_core_web_sm')
    def __call__(self, doc):
        nlpdoc = self.spacynlp(doc)
        nlpdoc = [token.lemma_.lower() for token in nlpdoc if (not token.is_punct)]
        return nlpdoc


In [6]:
def sorted_list2str(s): 
    str1 = "" 
    for ele in s: 
        str1 += " " + ele  
    return str1 

In [7]:
def lemmatized_txt2str(s):
    str1 = "" 
    for ele in s: 
        for ele2 in ele:
            if ele2.isspace():
                continue
            str1 += " " + ele2  
    return str1 

In [8]:
lemma_tokenizer = LemmaTokenizer()
def lemmatize(sentences):
    # Because of the TextRank algorithm, we have to split the document into sentences to create the document corpus 
    # (document corpus is the k most important sentences after applying TextRank algorithm)


    sentences = tokenizer.tokenize(sorted_list2str(sentences))
    sentences = [x for x in sentences if len(x) > NUM_OF_CHARACTERS]
    lemmatized_text = []

    for sentence in sentences:
        one_sentence = lemma_tokenizer(sentence)
        lemmatized_text.append(one_sentence)
    lemmatized_text = lemmatized_txt2str(lemmatized_text)
    return lemmatized_text

<h2> Apply TF-IDF </h2>

In [65]:
def call_vectorizer(df):
    tfidf_vectorizer = TfidfVectorizer(stop_words = "english")
    tfidf_vector = tfidf_vectorizer.fit_transform(df.iloc[:, 1].tolist())
    tfidf_df = pd.DataFrame(tfidf_vector.toarray(), columns=tfidf_vectorizer.get_feature_names())
        
    return tfidf_vectorizer, tfidf_df 

<h2> Global Term Frequency </h2>

To see how important a word is in the whole dataset, we calculate GTF_IDF matrix applying the formula below:

GTF_IDF = TF_IDF * sum(TF_IDF) / NUM_OF_DOCUMENTS

In [39]:

def calculate_gtfidf(tf_df):
    sum_of_idfs = tfidf_df.sum(axis = 0)
    for i in range(len(tfidf_df.columns)):
        tf_df[tfidf_df.columns[i]] = tf_df[tfidf_df.columns[i]].apply(lambda x: x * (sum_of_idfs[i] / NUM_OF_DOCUMENTS))
        
    return tf_df

<h2> The Experiment </h2>

We run the whole pipeline on N documents from the CourtListener database

In [25]:
df = pd.DataFrame(columns = ["id", "document"])
i = 0
for file_name in [file for file in os.listdir("data/train/") if file.endswith('.json')]:
    try:
        print(i)
        document = get_document("data/train/" + file_name)
        sorted_sentences = apply_textrank(document)
        df.loc[i] = lemmatize(sorted_sentences)
        df.loc[i] = [file_name, lemmatize(sorted_sentences)]
        i += 1
    except Exception as e:
        i += 1
        continue
  

0
1
2
2
2
3
4
5
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


D:\Fax\master\pravna informatika\SimilarLegalDocumentRetrieval\text_rank.py:99: RuntimeWarning: overflow encountered in double_scalars
  if abs(previous_pr - sum(pr_vector)) < self.min_diff:
D:\Fax\master\pravna informatika\SimilarLegalDocumentRetrieval\text_rank.py:102: RuntimeWarning: overflow encountered in double_scalars
  previous_pr = sum(pr_vector)


110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359


In [28]:
df

,id,document
0,174995.json,09 1504 united states of america appellee v. ...
1,174996.json,in this case the district court instruct the ...
2,175074.json,the bia affirm the april 3 2008 opinion of an...
3,175075.json,moreover fia 's affidavit explicitly confirm ...
4,175076.json,objection your honor to the line of questioni...
...,...,...
1391,198335.json,then on august 15 1994 toyota advise citi tha...
1392,198336.json,the court then say briefly that while petrone...
1393,198337.json,upon careful review of the record appellant '...
1394,198338.json,of medical examiners 375 u.s. 411 1964 we non...


In [30]:
df_test = pd.DataFrame(columns = ["id", "document"])
i = 0
for file_name in [file for file in os.listdir("data/test/") if file.endswith('.json')]:
    try:
        document = get_document("data/test/" + file_name)
        sorted_sentences = apply_textrank(document)
        df_test.loc[i] = lemmatize(sorted_sentences)
        df_test.loc[i] = [file_name, lemmatize(sorted_sentences)]
        i += 1
    except Exception as e:
        print(e)
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
'charmap' codec can't decode byte 0x98 in position 16801: character maps to <undefined>
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
'charmap' codec can't decode byte 0x98 in position 50298: character maps to <undefined>
181
182
183
184
185
186
187
188
189
190
191
192
'charmap' codec can't decode byte 0x98 in position 37913: character maps to <undefined>
192
'charmap' codec can't decode byte 0x98 in position 9838: chara

D:\Fax\master\pravna informatika\SimilarLegalDocumentRetrieval\text_rank.py:99: RuntimeWarning: invalid value encountered in double_scalars
  if abs(previous_pr - sum(pr_vector)) < self.min_diff:
D:\Fax\master\pravna informatika\SimilarLegalDocumentRetrieval\text_rank.py:102: RuntimeWarning: invalid value encountered in double_scalars
  previous_pr = sum(pr_vector)
D:\Fax\master\pravna informatika\SimilarLegalDocumentRetrieval\text_rank.py:98: RuntimeWarning: invalid value encountered in matmul
  pr_vector = (1 - self.damping) + self.damping * np.matmul(similarity_matrix, pr_vector)


224
225
226
227
228
229
230
231
'charmap' codec can't decode byte 0x98 in position 7885: character maps to <undefined>
231
232
'charmap' codec can't decode byte 0x98 in position 17479: character maps to <undefined>
232
233
234
'charmap' codec can't decode byte 0x98 in position 7292: character maps to <undefined>
234
235
236
237
238
239
240
'charmap' codec can't decode byte 0x98 in position 13332: character maps to <undefined>
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
'charmap' codec can't decode byte 0x98 in position 13773: character maps to <undefined>
281
282
283
284
'charmap' codec can't decode byte 0x98 in position 24319: character maps to <undefined>
284
285
286
287
288
289
290
291
292
293
294
295
296
297
'charmap' codec can't decode byte 0x98 in position 2586: character maps to <undefined>
297


D:\Fax\master\pravna informatika\SimilarLegalDocumentRetrieval\text_rank.py:98: RuntimeWarning: overflow encountered in matmul
  pr_vector = (1 - self.damping) + self.damping * np.matmul(similarity_matrix, pr_vector)


298
299
300
301
302
303
304
305
306
307
308
309
'charmap' codec can't decode byte 0x81 in position 935: character maps to <undefined>
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
'charmap' codec can't decode byte 0x98 in position 25505: character maps to <undefined>
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
'charmap' codec can't decode byte 0x98 in position 14198: character maps to <undefined>
361
362
363
364
365
366
367
'charmap' codec can't decode byte 0x81 in position 1151: character maps to <undefined>
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418


In [29]:
df.to_csv("train_textrank.csv", sep='\t')

In [31]:
df_test.to_csv("test_textrank.csv", sep='\t')

In [66]:
vectorizer, tfidf_df = call_vectorizer(df)
train_gtf_idf = calculate_gtfidf(tfidf_df)

    

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [67]:
train_gtf_idf.sum(axis = 0)

00          1.645593
000        16.412672
001         0.000114
001b        0.051524
005         0.000197
             ...    
zuleta      0.029197
zulma       0.002448
zuluaga     0.013546
zurosky     0.001466
zyrone      0.002997
Length: 24401, dtype: float64

In [81]:

def cos_similarity(): 
    results = pd.DataFrame(columns = ["verdict", "indexes"])
    for i, doc in enumerate(df_test.iloc[:, 1]): 
        tfidf_test = vectorizer.transform([doc])
        tfidf_test = pd.DataFrame(tfidf_test.toarray(), columns=vectorizer.get_feature_names())
        tfidf_test = calculate_gtfidf(tfidf_test)
        distances = cosine_similarity(tfidf_test, train_gtf_idf).flatten()
        indexes = np.argsort(distances)[::-1]
        indexes = indexes[:100]
        results = results.append({ 
            "verdict" : df_test.iloc[i, 0], 
            "indexes" : indexes}, ignore_index=True))
    return results

In [82]:
results = cos_similarity()

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

...

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python3

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [84]:
results

,0,indexes,verdict
0,198340.json,NaN,NaN
1,"[1213, 1180, 814, 1132, 853, 918, 696, 343, 11...",NaN,NaN
0,198341.json,NaN,NaN
1,"[680, 815, 991, 641, 779, 1277, 367, 79, 510, ...",NaN,NaN
0,198342.json,NaN,NaN
...,...,...,...
1,"[454, 1173, 1180, 836, 1126, 224, 545, 1159, 1...",NaN,NaN
0,199129.json,NaN,NaN
1,"[680, 510, 1317, 1277, 779, 815, 1093, 1262, 7...",NaN,NaN
0,199130.json,NaN,NaN


In [83]:
results.to_csv("results/text_rank.csv", sep = "\t")